Импортируем все нужные нам библиотеки.

In [1]:
import os
import glob
import random
from pathlib import Path
from datetime import datetime

import torch
import numpy as np
from PIL import Image
from sklearn.metrics import f1_score
from torchvision import datasets, transforms, models

SEED = 2005

Инициализируем генератор случайных чисел.

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

seed_everything(SEED)

Создадим трансформации изображений для тренировочной и валидационной выборок.

Изображения тренировочной выборки будем поворачивать, брать случайные фрагменты изображения, изменять яркость/контраст/насыщенность.

Для создания датасета воспользуемся встроенной библиотекой из пакета torchvision.

In [4]:
transforms_train = transforms.Compose([
    transforms.RandomRotation(8),
    transforms.RandomResizedCrop(224, scale=(0.90, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.1),
    transforms.ColorJitter(brightness=0.10, contrast=0.10, saturation=0.10, hue=0.03),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.RandomErasing(scale=(0.02, 0.10))]
)

transforms_valid = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
)

dataset_train = datasets.ImageFolder('train', transform = transforms_train)
dataset_valid = datasets.ImageFolder('train', transform = transforms_valid)

dataset_train, dataset_valid

(Dataset ImageFolder
     Number of datapoints: 18834
     Root location: train
     StandardTransform
 Transform: Compose(
                RandomRotation(degrees=[-8.0, 8.0], interpolation=nearest, expand=False, fill=0)
                RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.1)
                ColorJitter(brightness=(0.9, 1.1), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.03, 0.03))
                PILToTensor()
                ConvertImageDtype()
                Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0, inplace=False)
            ),
 Dataset ImageFolder
     Number of datapoints: 18834
     Root location: train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, 

Посмотрим на наши классы изображений.

In [9]:
dataset_train.classes

['0', '1']

Возьмем предобученную модель VisionTransformer, заменим в ней классификационный слой и инициализируем его.

In [6]:
model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_SWAG_LINEAR_V1)
model.heads.head = torch.nn.Linear(768, len(dataset_train.classes))

torch.nn.init.xavier_uniform_(model.heads.head.weight)
torch.nn.init.constant_(model.heads.head.bias, 0.0)
model.to('cuda')

model.heads

Sequential(
  (head): Linear(in_features=768, out_features=2, bias=True)
)

Создадим даталоадеры, которые будут бить данные на батчи и отдавать модели.


In [7]:
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=32,
                                           num_workers=4, shuffle=True,
                                           drop_last=True, pin_memory=True)
loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=32,
                                           num_workers=4, shuffle=False,
                                           drop_last=False, pin_memory=True)

Так как классы не сбалансированы, то для функции потерь посчитаем корректировочные коэффициенты.

In [8]:
weight = len(dataset_train.targets) / (len(np.unique(dataset_train.targets)) * np.bincount(dataset_train.targets))
weight = torch.FloatTensor(weight)
weight = torch.nan_to_num(weight, posinf=1.0, neginf=1.0)
weight = weight.to('cuda') / weight.min()
weight

tensor([1.0000, 1.0189], device='cuda:0')

Замораживаем предобученные слои и одну эпоху учим только наш классификационный слой. 

In [9]:
%%time
# pretrain

for param in model.parameters():
    param.requires_grad = False

for param in model.heads.parameters():
    param.requires_grad = True

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-7)
criterion = torch.nn.CrossEntropyLoss(weight)

model.train()
optimizer.zero_grad()
for imgs, label in loader_train:
    pred = model(imgs.to('cuda'))
    loss = criterion(pred, label.to('cuda'))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    optimizer.zero_grad()

CPU times: user 1min 31s, sys: 3 s, total: 1min 34s
Wall time: 1min 29s


Основной цикл вычислений. Размораживаем все слои и учим модель 15 эпох.
В качестве шедулера возьмем OneCycleLR - он плавно повышает LR до целевого значения и потом сильно понижает его.
Во время валидации проверяем качество метрики F1. Если метрика не повышается несколько эпох, то загружаем в модель предыдущие лучшие веса.

In [ ]:
epochs = 6

for param in model.parameters():
    param.requires_grad = True

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
criterion = torch.nn.CrossEntropyLoss(weight)

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, epochs=epochs, max_lr=5e-6,
                                                   div_factor=10.0, final_div_factor=10.0,
                                                   steps_per_epoch=1)

best_f1  = 0
best_cnt = 0

for epoch in range(epochs):
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Start epoch {epoch + 1} at {datetime.now().strftime('%H:%M:%S')}, lr={current_lr:0.8f}")
    model.train()
    optimizer.zero_grad()
    train_losses = []
    for imgs, label in loader_train:
        pred = model(imgs.to('cuda'))
        loss = criterion(pred, label.to('cuda'))
        loss.backward()
        train_losses.append(loss.item())
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

    lr_scheduler.step()

    model.eval()
    torch.cuda.empty_cache()

    val_label = []
    trg_label = []
    with torch.no_grad():
        for imgs, label in loader_valid:
            pred = model(imgs.to('cuda'))
            trg_label.extend(label.numpy().tolist())
            val_label.extend(pred.argmax(dim=1).cpu().numpy().flatten().tolist())
    f1_label = f1_score(trg_label, val_label, zero_division=0, average='macro')
    
    print(f"  train loss: {np.mean(train_losses):6.4f} valid f1: {f1_label:6.4f}")

    if f1_label > best_f1:
        best_f1 = f1_label
        best_cnt = 0
        torch.save(model.state_dict(), "model.pth")
        print("Saved best model!")
    else:
        best_cnt += 1

    if best_cnt > 2:
        print("Loading best model weights!")
        model.load_state_dict(torch.load("model.pth"))

Start epoch 1 at 17:51:38, lr=0.00000050
  train loss: 0.0336 valid f1: 0.9909
Saved best model!
Start epoch 2 at 17:56:45, lr=0.00000497
  train loss: 0.0312 valid f1: 0.9998
Saved best model!
Start epoch 3 at 18:01:52, lr=0.00000407
  train loss: 0.0066 valid f1: 1.0000
Saved best model!
Start epoch 4 at 18:06:58, lr=0.00000234
  train loss: 0.0004 valid f1: 1.0000
Start epoch 5 at 18:12:04, lr=0.00000071


In [ ]:
torch.save(model.state_dict(), "model_LB_xx.pth")

In [ ]:
rev_idx0 = {v:k for k, v in dataset_train.class_to_idx.items()}
rev_idx0